<a href="https://colab.research.google.com/github/JinFree/darknet/blob/master/Train_DarkNet_Yolov4_tiny_VOC_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

별도로 복사해 둔 DarkNet 저장소를 받아온 후 빌드하겠습니다.
nvidia-smi를 통해 나오는 GPU Name을 확인해두어야 합니다.

In [ ]:
!git clone https://github.com/JinFree/darknet.git
!cp darknet/Makefile_Colab darknet/Makefile 
!nvidia-smi

DarkNet을 빌드합니다.
만약 빌드 중 문제가 생기면 Makefile의 20번 줄 ARCH을 확인한 후 위에서 확인한 GPU 이름의 CUDA Compute Capability를 확인하여 수정해주시면 되겠습니다.

In [ ]:
!cd darknet && make -j$(nproc)

아래 셀을 실행했을 때 bycycle, person, dog, truck, car에 대한 객체 인식 결과를 확인할 수 있다면 DarkNet 빌드가 정상적으로 수행 된 것입니다.

아래와 유사하게 출력됩니다.

----

data/dog.jpg: Predicted in 5.928000 milli-seconds.

bicycle: 29%

person: 25%

dog: 72%

truck: 82%

car: 46%

----


In [ ]:
!cd darknet \
&& ./darknet detect cfg/yolov4-tiny.cfg bin/yolov4-tiny.weights data/dog.jpg

데이터셋을 준비하겠습니다. 약 5분이 소요됩니다.

In [ ]:
!wget http://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar
!tar -xf VOCtrainval_11-May-2012.tar
!rm VOCtrainval_11-May-2012.tar

다운받은 데이터셋은 Pascal VOC 데이터셋입니다. 이를 DarkNet에서 훈련할 때 사용하기 위해서는 xml 파일로 되어있는 라벨 파일을 txt 파일로 바꿔주어야 합니다.

이를 수행할 수 있는 깃헙 저장소를 받은 후 실행하겠습니다.

In [ ]:
!git clone https://github.com/ssaru/convert2Yolo.git
!cp darknet/vocnames.txt ./vocnames.txt
!mkdir VOCdevkit/VOC2012/labels/
!cd convert2Yolo && python3 example.py --datasets VOC \
--img_path ../VOCdevkit/VOC2012/JPEGImages/ \
--label ../VOCdevkit/VOC2012/Annotations/ \
--convert_output_path ../VOCdevkit/VOC2012/labels/ \
--img_type ".jpg" \
--manifest_path ../ \
--cls_list_file ../vocnames.txt

Yolov4-tiny 훈련을 위해 전이학습할 가중치를 받아온 후 VOC 데이터셋을 학습할 수 있도록 수정한 yolov4-tiny_voc.cfg 파일을 준비합니다.

아래는 yolov4-tiny_voc.cfg 파일에 대한 설명입니다.

----

해당 파일 내 있는 두 개의 yolo 레이어에서 클래스 수를 20으로 수정했습니다.

이후 yolo레이어 바로 위의 convolutional 레이어에서 filters 수를 75로 수정했습니다.

75는 ( 5 + 20 ) * 3을 통해 계산되었습니다.

----

In [ ]:
!mkdir backup && cd backup && wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29
!cp darknet/yolov4-tiny_voc.cfg ./yolov4-tiny_voc.cfg

훈련을 위한 vocdata_colab.txt 파일을 준비합니다.

훈련할 클래스의 수, 훈련할 데이터셋의 리스트가 적힌 파일의 경로, 클래스의 이름, 훈련 중 가중치가 저장될 폴더의 경로가 아래와 같이 적혀있습니다.

---

classes = 20

train = /content/manifest.txt

names = /content/vocnames.txt

backup = /content/backup

---

In [8]:
!cp darknet/vocdata_colab.txt ./vocdata_colab.txt

훈련을 수행합니다.

훈련은 약 두시간정도 수행하면 됩니다.

훈련하면서 출력되는 로그는 train_log.txt로 저장됩니다.

In [ ]:
!cd darknet && ./darknet detector train ../vocdata_colab.txt ../yolov4-tiny_voc.cfg ../backup/yolov4-tiny.conv.29 -dont_show  2&> train_log.txt

훈련이 어느정도 수행되면 훈련을 중지하셔도 됩니다.

직접 추론하기 위해서 필요한 파일은 아래의 네 가지 파일입니다.

----

darknet/vocdata.txt

vocnames.txt

yolov4-tiny_voc.cfg

backup/yolov4-tiny_voc_last.weights

----

만약 추가적으로 학습을 하고자 하는 경우, 아래의 셀을 실행하면 됩니다.

In [ ]:
!cd darknet && ./darknet detector train ../vocdata_colab.txt ../yolov4-tiny_voc.cfg ../backup/yolov4-tiny_voc_last.weights -dont_show  2&> train_log.txt